In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math 

In [ ]:
def checkOutliers(dfColumn, maxValue, greater):
  if greater:
    (dfColumn > maxValue).any()
  else:
    (dfColumn < maxValue).any()

def deleteOutliers(df, dfRow, value, greater):
  if greater:
    df.drop(df[(dfRow > value)].index, inplace = True)
  else:
    df.drop(df[(dfRow < value)].index, inplace = True)

def getDataFrame(index, explanation_type=''):
  dfs = None
  warmup = 10
  for y in range(50, 201, 50):
    df = pd.read_csv(f'Question{index}Length{y}Explanation{explanation_type}.csv', skipinitialspace=True).iloc[warmup:, :]
    dfs = pd.concat([dfs,df]) 
  dfs_mean_by_index = dfs.groupby("Index").mean()
  #deleteOutliers(dfs_mean_by_index, dfs_mean_by_index.Memory, 0, False)
  #deleteOutliers(dfs_mean_by_index, dfs_mean_by_index.Time, 20, True)
  return dfs_mean_by_index, dfs

def roundValues(dfColumn):
  print(math.ceil(dfColumn).mean())
  return math.ceil(dfColumn).mean()

def createPlot(d1, d2, labelLine, labelx, labely):
  plt.plot(d1, d2, label = labelLine)
  plt.xlabel(labelx)
  plt.ylabel(labely)
  plt.legend()
  plt.show()

# BW general explanation data set

*   load csv(s)
  *   drop first elements: warmup phase 
  *   drop memory's outliers: replace negative values with 0 (garbage collector?)
  *   drop time's outliers: replace values >30 ms with 30 (idk)
*   merge csv
  *   concat referring to the same question
  *   calculate mean of the element of the same index (application of the same operation on plan of different length)





In [ ]:
questionsList = ["Q1", "Q2", "Q3", "Q4", "Q5"]
questions = pd.DataFrame(questionsList)

means_by_index_q1, dfQ1 = getDataFrame(1)
means_by_index_q2, dfQ2 = getDataFrame(2)
means_by_index_q3, dfQ3 = getDataFrame(3)
means_by_index_q4, dfQ4 = getDataFrame(4)
means_by_index_q5, dfQ5 = getDataFrame(5)

dfs = [dfQ1,dfQ2,dfQ3,dfQ4, dfQ5]
dfs_merged = pd.concat(dfs)
means_by_index = [means_by_index_q1, means_by_index_q2, means_by_index_q3, means_by_index_q4, means_by_index_q5]
means_memory_general_explanation = [math.ceil(means_by_index_q1.Memory.mean()), math.ceil(means_by_index_q2.Memory.mean()), math.ceil(means_by_index_q3.Memory.mean()), math.ceil(means_by_index_q4.Memory.mean()), math.ceil(means_by_index_q5.Memory.mean())]
means_time_general_explanation = [math.ceil(means_by_index_q1.Time.mean()), math.ceil(means_by_index_q2.Time.mean()), math.ceil(means_by_index_q3.Time.mean()), math.ceil(means_by_index_q4.Time.mean()), math.ceil(means_by_index_q5.Time.mean())]

# BW contrastive explanation dataset

*   load csv(s)
  *   drop first elements: warmup phase 
  *   drop memory's outliers: replace negative values with 0 (garbage collector?)
*   merge csv
  *   concat referring to the same question
  *   calculate mean of the element of the same index (application of the same operation on plan of different length)
  

In [ ]:
means_by_index_q1C, dfQ1C = getDataFrame(1, 'c')
means_by_index_q2C, dfQ2C = getDataFrame(2, 'c')
means_by_index_q3C, dfQ3C = getDataFrame(3, 'c')
means_by_index_q4C, dfQ4C = getDataFrame(4, 'c')
means_by_index_q5C, dfQ5C = getDataFrame(5, 'c')

dfsC = [dfQ1C,dfQ2C,dfQ3C,dfQ4C, dfQ5C]
dfs_mergedC = pd.concat(dfsC)
means_by_indexC = [means_by_index_q1C, means_by_index_q2C, means_by_index_q3C, means_by_index_q4C, means_by_index_q5C]
means_memory_contrastive_explanation = [math.ceil(means_by_index_q1C.Memory.mean()), math.ceil(means_by_index_q2C.Memory.mean()), math.ceil(means_by_index_q3C.Memory.mean()), math.ceil(means_by_index_q4C.Memory.mean()), math.ceil(means_by_index_q5C.Memory.mean())]
means_time_contrastive_explanation = [math.ceil(means_by_index_q1C.Time.mean()), math.ceil(means_by_index_q2C.Time.mean()), math.ceil(means_by_index_q3C.Time.mean()), math.ceil(means_by_index_q4C.Time.mean()), math.ceil(means_by_index_q5C.Time.mean())]

In [ ]:
dfQ5.Memory

10     3145728
11     2097152
12     2097152
13     3145728
14     2097152
        ...   
195    3145728
196    3145728
197    3145728
198    3145728
199    3145728
Name: Memory, Length: 460, dtype: int64

# Create aggregate results

In [ ]:
def createDfPlans(df):
  listTime = []
  listMemory = []
  for y in range(50, 201, 50):
    listTime.append(df[(df.PlanLength == y)].Time.mean())
    listMemory.append(df[(df.PlanLength == y)].Memory.mean())
    print()
  return listMemory, listTime


def init(memory, time):
  means_memory_explanation = pd.DataFrame(memory, columns=['Memory(Byte)'])
  means_time_explanation =  pd.DataFrame(time, columns=['Time(ms)'])
  return pd.concat([means_memory_explanation, means_time_explanation], axis=1)

def write(question, means_general_explanation, means_contrastive_explanation, fileName): 
  means = {'Explanation': question, 'General' : means_general_explanation, 'Contrastive': means_contrastive_explanation}
  means = pd.concat(means.values(), axis=1, keys=means.keys())
  means.rename(columns={0: 'Length'}, inplace=True)
  means.to_csv('{}.csv'.format(fileName))
  print(means)

def writeResult(dfQ, dfQC, fileName):
  i, j = createDfPlans(dfQ)
  means_general_explanation = init(i,j)
  i, j = createDfPlans(dfQC)
  means_contrastive_explanation = init(i, j)
  write(pd.DataFrame(["50", "100", "150", "200"]), means_general_explanation, means_contrastive_explanation, fileName)

def writeAggregateResult(filename):
  memory_general_explanation = pd.DataFrame(means_memory_general_explanation, columns=['Memory(Byte)'])
  time_general_explanation =  pd.DataFrame(means_time_general_explanation, columns=['Time(ms)'])

  memory_contrastive_explanation = pd.DataFrame(means_memory_contrastive_explanation, columns=['Memory(Byte)'])
  time_contrastive_explanation = pd.DataFrame(means_time_contrastive_explanation, columns=['Time(ms)'])

  means_general_explanation = pd.concat([memory_general_explanation, time_general_explanation], axis=1)
  means_contrastive_explanation = pd.concat([memory_contrastive_explanation, time_contrastive_explanation], axis=1)
  write(pd.DataFrame(["Q1", "Q2", "Q3", "Q4", "Q5"]), means_general_explanation, means_contrastive_explanation, filename)

writeResult(dfQ1, dfQ1C, "q1")
writeResult(dfQ2, dfQ2C, "q2")
writeResult(dfQ3, dfQ3C, "q3")
writeResult(dfQ4, dfQ4C, "q4")
writeResult(dfQ5, dfQ5C, "q5")

writeAggregateResult("aggregate")

#means_general_explanation_bw.reorder_levels([1,0], axis=1)









  Explanation       General             Contrastive          
       Length  Memory(Byte)  Time(ms)  Memory(Byte)  Time(ms)
0          50  1.703936e+06  7.025000  1.074790e+06  5.900000
1         100  1.433054e+06  4.866667  1.118481e+06  4.277778
2         150  1.685211e+06  3.007143  1.101005e+06  2.664286
3         200  1.098245e+06  2.310526  1.076170e+06  2.694737








  Explanation       General             Contrastive          
       Length  Memory(Byte)  Time(ms)  Memory(Byte)  Time(ms)
0          50  2.909798e+06  5.200000  3.040870e+06  4.325000
1         100  3.064172e+06  3.833333  3.134077e+06  3.933333
2         150  2.082172e+06  3.392857  3.145728e+06  3.378571
3         200  2.626959e+06  2.752632  2.792523e+06  3.036842








  Explanation       General             Contrastive          
       Length  Memory(Byte)  Time(ms)  Memory(Byte)  Time(ms)
0          50  3.617587e+06  5.400000  2.306867e+06  4.200000
1         100  3.821477e+06  3.666667  2.94766